### Runpod 설정

In [ ]:
import os

# 모델 저장 경로를 네트워크 볼륨(/workspace) 내부로 지정
os.environ["HF_HOME"] = "/workspace/hf_cache"

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install transformers accelerate bitsandbytes openai torch nltk huggingface_hub bert-score

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

---

In [7]:
script = "상담사: 카드 ▲▲▲입니다.\n손님: 네, 안녕하세요?\n상담사: 네, 안녕하세요?\n손님: 카드 네, 카드 그\n손님: 돈 빠져 나가는 날짜 있잖아요?\n상담사: 네.\n손님: 그것 좀 변경하고 싶은데요\n상담사: 네, 그러십니까? 카드로 보험료 내시는 날짜 말씀이실까요?\n손님: 보험료가 아니라 저기 뭐지 어 카드 사용하면\n상담사: 네,\n손님: 카드 사용하면 그 지출하는 날짜가 있잖아요?\n상담사: 아 네 네, 카드 대금 내시는 날짜 결제 일자 말씀 말씀하시는거죠?\n손님: 네.\n상담사: 네 네, 그러면 정보 확인 후에 제가 조회 후에 변경 도와드릴 텐데요, 휴대폰 번호와 생년월일 말씀해 주세요.\n손님: ▲▲▲▲▲▲▲▲▲▲\n상담사: 네.\n손님: ▲▲▲▲▲▲\n상담사: 네, 본인 성함이 어떻게 되십니까? 결제 은행은 어느 은행 이용 중이실까요?\n손님: ▲▲▲\n손님: ▲▲\n상담사: 네, 소중한 정보 확인 감사합니다. 지금은 그러면 ▲▲일부터 말일까지 쓰신 내역을 ▲▲일날 내시는 걸로 되어있는데 그럼 며칠로 변경하시려는 걸까요?\n손님: 예, 네 네.\n손님: 아 네, ▲▲일이요.\n상담사: 네, ▲▲일로 변경하시게 되시면 이제 원래는 ▲▲일부터 말일까지 쓰신 내역을 카드 대금 내셨는데 ▲▲일은 좀 달라요. 매월 월 ▲▲일부터 ▲▲일까지 사용하신 게 ▲▲일 날 청구가 되신다는 점 참고 부탁드리고요. 결제일\n손님: 네.\n상담사: 변경하시고 ▲▲▲▲내에는 재변경이 어려울 수 있는데 괜찮으실까요?\n손님: 네.\n상담사: 결제일 변경하시고 ▲▲▲내에는 또 다시 변경하시는 게 어려우실 수 있는데 괜찮으실까요? 결제일 변경해도 기존 이용 금액이 변경된 결제일에 청구 되지 않을 수도 있으며 단기 카드 대출이나 자동 이체의 경우 이용 기간에 따라 두 달\n손님: 아 아 네 네.\n상담사: 청구되실 수 있다는 점 참고 부탁드리고요, 변경하신 결제일에 대한 이용 기간 한 번 더 문자로 발송해드릴테니 꼭 내용 확인해 보시고 이용 부탁드리겠습니다. 오늘은 ▲▲월부터 지금 변경되신 ▲▲일 날짜에 납부하시게 될 거니 그 부분도 참고 부탁드리겠습니다.\n손님: 네.\n상담사: 감사합니다. 완료되셨습니다.\n손님: 네.\n상담사: 네, 상담사 ▲▲▲이었습니다.\n손님: 네."

In [ ]:
prompt = f"""
상담 스크립트에서 고객(손님)의 발화만을 기준으로 감정을 분석하세요

### 분석 규칙
1. 상담원 발화는 완전히 무시하고, 고객 발화만 사용합니다
2. 전체 상담 스크립트를 시간 흐름 기준으로 3등분하여 감정을 판단합니다
3. 각 구간마다 감정을 3가지 중 반드시 하나만 선택합니다 - 부정 | 중립 | 긍정
4. 추측, 설명, 해설 문장 금지
5. JSON 외의 서론, 결론, 마크다운 기호(```), ```json\n은 절대 포함하지 마세요

### 감정 판단 기준
- 부정: 불만, 분노, 짜증, 불안, 항의, 문제 제기
- 중립: 사실 전달, 질문, 감정 표현 거의 없음
- 긍정: 만족, 안도, 감사, 동의, 긍정적 반응

### 상담 스크립트
{script}

### 출력 형식(JSON 고정)
{{
  "early": "부정|중립|긍정",
  "mid": "부정|중립|긍정",
  "late": "부정|중립|긍정"
}}
"""

---
### GPT

In [3]:
import time
import json
import openai

In [4]:
def generate_feedback_openai(model_name, prompt):
    start_time = time.perf_counter()
    ttft = 0
    generated_tokens = 0
    content = ""
    first_token_received = False
    
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            response_format={"type": "json_object"},
            stream=True,                              # 토큰 단위로 수신
            stream_options={"include_usage": True}    # 마지막 chunk에 토큰 정보 포함
        )
        
        for chunk in response:
            # 첫번째 토큰이 들어오는 시점 확인 (ttft)
            if not first_token_received and chunk.choices and chunk.choices[0].delta.content:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            # 내용 누적
            if chunk.choices and chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content
            
            # 토큰 사용량 확인
            if chunk.usage is not None:
                generated_tokens = chunk.usage.completion_tokens
        
        end_time = time.perf_counter()
        total_duration = end_time - start_time
        
        # tps 계산 (생성된 토큰 수 / 전체 소요 시간)
        tps = generated_tokens / total_duration if total_duration > 0 else 0

        result_json = json.loads(content)
        
        metrics = {
                "ttft": round(ttft, 3),
                "tps": round(tps, 2),
                "total_tokens": generated_tokens
            }

        return result_json, metrics
    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", "오류"

In [8]:
res_gpt_sm, metrics_gpt_sm = generate_feedback_openai("gpt-4.1-mini", prompt)

In [9]:
res_gpt_sm

{'초반': '중립', '중반': '중립', '후반': '긍정'}

In [10]:
metrics_gpt_sm

{'ttft': 0.406, 'tps': 37.2, 'total_tokens': 30}

---

### sllm

In [ ]:
import re
import json
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

In [ ]:
def generate_summary_sllm(model_name, prompt):
    try:
        # 모델 및 토크나이저 로드
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        messages = [{"role": "system", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages, 
            tokenize=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)

        # 스트리머 설정
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
        # 측정 변수 초기화
        start_time = time.perf_counter()
        ttft = 0
        first_token_received = False
        full_content = ""
        generated_token_count = 0

        # 생성 프로세스를 별도 스레드에서 실행
        generation_kwargs = dict(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=512,
            repetition_penalty=1.2,
            do_sample=False,
            temperature=0.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        # 토큰을 하나씩 읽으며 ttft 측정
        for new_text in streamer:
            if not first_token_received and len(new_text) > 0:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            full_content += new_text

        thread.join()
        end_time = time.perf_counter()
        
        total_duration = end_time - start_time
        generated_token_count = len(tokenizer.encode(full_content))
        tps = generated_token_count / total_duration if total_duration > 0 else 0

        # JSON 추출 및 파싱
        try:
            json_match = re.search(r'(\{.*\})', full_content, re.DOTALL)
            if json_match:
                content = json.loads(json_match.group(1))
            else:
                content = {"error": "JSON 형식을 찾을 수 없음", "raw": full_content}
        except json.JSONDecodeError:
            content = {"error": "JSON 파싱 실패", "raw": full_content}

        # 리소스 정리
        del model, tokenizer, inputs
        torch.cuda.empty_cache()

        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": generated_token_count
        }

        return content, metrics

    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", None

In [ ]:
res, metrics = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", prompt)
# res, metrics = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", prompt)
# res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", prompt)
# res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct", prompt)
# res, metrics = generate_summary_sllm("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B", prompt)

In [ ]:
res

In [ ]:
import os
import json
import glob

# 파일 경로 설정
testset_dir = "./testsets"
json_files = glob.glob(os.path.join(testset_dir, "*.json"))

results_list = []

# 파일 루프 시작
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)
        
    for item in data_list:
        # 데이터 추출
        script = item.get('consulting_content', "")
        summary_ref = item['instructions'][0]['data'][0]['output']

        print(f"테스트 실행 중: {item.get('source_id', 'Unknown ID')}")

        # 모델 테스트
        # prompt 생성
        system_prompt = f"""
        상담 스크립트를 평가 기준에 따라 객관적으로 평가하세요

        ### 제약 사항
        1. 모든 감점에는 구체적인 발화 근거를 반드시 제시한다
        2. 고객의 감사 표현은 고객 발화에서만 카운트한다
        3. 추측, 설명 문장, 자연어 해설 금지 - JSON만 출력한다
        4. 총 점수는 60점이며 각 2개 점수의 총합이다

        ### 상담 스크립트
        {script}

        ### 평가 기준
        1. 매뉴얼 준수 (50점에서 감점하는 방식)
        intro
        - 인사말
        0점: 첫인사 + 마무리 멘트 모두 수행
        -5점: 첫인사 또는 마무리 멘트 누락
        - 고객확인
        0점: 고객정보를 고객에게 직접 질문
        -5점: 상담원이 고객정보를 먼저 발화하여 정보 누출

        response
        - 호응어
        0점: 공감/감성 호응
        -5점: 기운 없음, 짜증 섞인 표현
        - 대기 표현
        0점: 대기 표현 모두 수행
        -5점: 대기 표현 누락

        explanation
        - 커뮤니케이션
        0점: 핵심 요약 + 이해 쉬운 설명
        -5점: 일방적 설명, 단답형
        - 알기 쉬운 설명
        0점: 고객 눈높이 설명 + 부연
        -5점: 복잡한 설명/상담자 관점 설명

        proactivity
        - 적극성
        0점: 적극적 대응
        -5점: 수동적 대응, 대안 없음
        - 언어표현
        0점: 정중/경어체/긍정 표현
        -5점: 전문용어, 줄임말, 명령조, 무시 표현

        accuracy
        - 정확한 업무처리
        0점: 오류 없음
        -10점: 임의 판단으로 업무 오류 발생

        2. 고객 감사 표현 (10점)
        - 고객 발화 중 감사/칭찬 키워드 포함 시 1회 카운트
        - 0회: 0점 / 1회: 5점 / 2회 이상: 10점


        ### 출력 형식 (JSON)
        {{
        "manual_compliance": {{
            "intro": {{
            "score": 0,
            "evidence": []
            }},
            "response": {{
            "score": 0,
            "evidence": []
            }},
            "explanation": {{
            "score": 0,
            "evidence": []
            }},
            "proactivity": {{
            "score": 0,
            "evidence": []
            }},
            "accuracy": {{
            "score": 0,
            "evidence": []
            }},
            "manual_score": "0~50점"
        }},
        "customer_thanks": {{
            "count": 0,
            "thanks_score": "0~10점",
            "evidence": []
        }},
        "final_score": {{
            "total": "manual_score + thanks_score",
            "feedback": ""
        }}
        }}
        """
        
        res, metrics = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", system_prompt)
        # res, metrics = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", system_prompt)
        # res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", system_prompt)
        # res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct", system_prompt)
        # res, metrics = generate_summary_sllm("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B", system_prompt)
        # res, metrics = generate_feedback_openai("gpt-4.1-mini", system_prompt)
            
        # 결과 저장
        results_list.append({
            "id": item.get('source_id'),
            "res": res,
            "metrics": metrics,
        })

테스트 실행 중: 20593
테스트 실행 중: 20594
테스트 실행 중: 20596
테스트 실행 중: 20597
테스트 실행 중: 20598
테스트 실행 중: 20599
테스트 실행 중: 20600
테스트 실행 중: 20601
테스트 실행 중: 20602
테스트 실행 중: 20603
테스트 실행 중: 20604
테스트 실행 중: 20605
테스트 실행 중: 20606
테스트 실행 중: 20607
테스트 실행 중: 20608
테스트 실행 중: 20609
테스트 실행 중: 20610
테스트 실행 중: 20611
테스트 실행 중: 20612
테스트 실행 중: 20613
테스트 실행 중: 20614
테스트 실행 중: 20615
테스트 실행 중: 20616
테스트 실행 중: 20617
테스트 실행 중: 20618
테스트 실행 중: 20619
테스트 실행 중: 20620
테스트 실행 중: 20621
테스트 실행 중: 20622
테스트 실행 중: 20623
테스트 실행 중: 20624
테스트 실행 중: 20625
테스트 실행 중: 20626
테스트 실행 중: 20627
테스트 실행 중: 20628
테스트 실행 중: 20629
테스트 실행 중: 20630
테스트 실행 중: 20631
테스트 실행 중: 20632
테스트 실행 중: 20633
테스트 실행 중: 20634
테스트 실행 중: 20635
테스트 실행 중: 20636
테스트 실행 중: 20637
테스트 실행 중: 20638
테스트 실행 중: 20639
테스트 실행 중: 20640
테스트 실행 중: 20641
테스트 실행 중: 20642
테스트 실행 중: 20643
테스트 실행 중: 20644
테스트 실행 중: 20645
테스트 실행 중: 20646
테스트 실행 중: 20647
테스트 실행 중: 20648
테스트 실행 중: 20649
테스트 실행 중: 20650
테스트 실행 중: 20651
테스트 실행 중: 20652
테스트 실행 중: 20653
테스트 실행 중: 20654
테스트 실행 중: 20655
테스트 실행 중

In [ ]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)
df_new['model'] = "kanana-nano-2.1b" 

output_file = "evaluation_results_feedback.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: evaluation_results_feedback.csv
